In [ ]:
import numpy as np
from scipy.stats import lognorm, norm


def mode_price(S, mu, sigma, T, t):
    tau = T - t
    mode_S_T = S * np.exp((mu - (3 / 2) * sigma ** 2) * tau)
    return mode_S_T


def black_scholes_call_with_dividend_and_d_values(S, K, r, sigma, T, t, delta):
    tau = T - t
    d1 = (np.log(S / K) + (r - delta + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    C = S * np.exp(-delta * tau) * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    return C, d1, d2


def black_scholes_put_with_dividend_and_d_values(S, K, r, sigma, T, t, delta):
    tau = T - t
    d1 = (np.log(S / K) + (r - delta + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    P = K * np.exp(-r * tau) * norm.cdf(-d2) - S * np.exp(-delta * tau) * norm.cdf(-d1)
    return P, d1, d2


def stock_price_probability(S, sigma, mu, T, t, value, direction):
    tau = T - t
    mu_adjusted = np.log(S) + (mu - (1 / 2) * sigma ** 2) * tau
    sigma_adjusted = sigma * np.sqrt(tau)

    dist = lognorm(s=sigma_adjusted, scale=np.exp(mu_adjusted))
    if direction == 'greater':
        return 1 - dist.cdf(value)
    elif direction == 'less':
        return dist.cdf(value)
    else:
        raise ValueError("The direction parameter should be 'greater' or 'less'.")


def calculate_greeks(S, K, r, sigma, T, t, d1, d2):
    tau = T - t
    n_d1 = norm.pdf(d1)
    n_d2 = norm.pdf(d2)

    # Greeks for Call options
    delta_call = norm.cdf(d1)
    gamma_call = n_d1 / (S * sigma * np.sqrt(tau))
    vega_call = S * n_d1 * np.sqrt(tau)  # Vega is typically quoted per 1% (0.01) change in volatility,,but here we don't. To do so, multiply by 0.01.
    theta_call = (-S * n_d1 * sigma / (2 * np.sqrt(tau)) - r * K * np.exp(-r * tau) * norm.cdf(d2)) #To calculate daily theta, divide by 365.
    rho_call = K * tau * np.exp(-r * tau) * norm.cdf(d2) # Rho is typically quoted per 1% (0.01) change in rates,but here we don't. To do so, multiply by 0.01.

    # Greeks for Put options
    delta_put = norm.cdf(d1) - 1
    gamma_put = gamma_call  # Gamma is the same for calls and puts.
    vega_put = vega_call  # Vega is the same for calls and puts.
    theta_put = (-S * n_d1 * sigma / (2 * np.sqrt(tau)) + r * K * np.exp(-r * tau) * norm.cdf(-d2)) #To calculate daily theta, divide by 365.
    rho_put = -K * tau * np.exp(-r * tau) * norm.cdf(
        -d2) # Rho is typically quoted per 1% (0.01) change in rates,but here we don't. To do so, multiply by 0.01.

    return delta_call, gamma_call, vega_call, theta_call, rho_call, delta_put, gamma_put, vega_put, theta_put, rho_put


def financial_calculations_input():
    S = float(input("Enter the current stock price S(t): "))
    K = float(input("Enter the strike price K: "))
    r = float(input("Enter the annual risk-free interest rate r: "))
    sigma = float(input("Enter the volatility (standard deviation of annual stock returns) σ: "))
    T = float(input("Enter the maturity time T (in years): "))
    t = float(input("Enter the current time t (in years): "))
    delta = float(input("Enter the continuous dividend rate δ: "))
    mu = float(input("Enter the average annual growth rate of the stock price μ: "))
    value = float(input("Enter the value (strike price) to compare the stock price against: "))
    direction = input(
        "Calculate the probability of the stock price being 'greater' (call ITM) or 'less' (put ITM) than the compared value: ").lower()
    calc_type = input("Calculate 'call/put', 'mode', 'probability', 'greeks', or 'all'?: ").lower()

    # Calculate d1, d2 for Greek calculations
    tau = T - t
    d1 = (np.log(S / K) + (r - delta + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)

    if calc_type in ['call/put', 'all']:
        C, d1, d2 = black_scholes_call_with_dividend_and_d_values(S, K, r, sigma, T, t, delta)
        P, _, _ = black_scholes_put_with_dividend_and_d_values(S, K, r, sigma, T, t, delta)
        print(f"The Black-Scholes call option price is: {C}")
        print(f"The Black-Scholes put option price is: {P}")
        print(f"The value of d1 is: {d1}")
        print(f"The value of d2 is: {d2}")

    if calc_type in ['mode', 'all']:
        mode_price_val = mode_price(S, mu, sigma, T, t)
        print(f"The MODE future stock price S(T) is: {mode_price_val}")

    if calc_type in ['probability', 'all']:
        prob = stock_price_probability(S, sigma, mu, T, t, value, direction)
        print(f"The probability of the stock price being {direction} than {value} at time T is: {prob}")

    if calc_type in ['greeks', 'all']:
        delta_call, gamma_call, vega_call, theta_call, rho_call, delta_put, gamma_put, vega_put, theta_put, rho_put = calculate_greeks(
            S, K, r, sigma, T, t, d1, d2)
        print(f"Call option Greeks:")
        print(f"Delta: {delta_call}, Gamma: {gamma_call}, Vega: {vega_call}, Theta: {theta_call}, Rho: {rho_call}")
        print(f"Put option Greeks:")
        print(f"Delta: {delta_put}, Gamma: {gamma_put}, Vega: {vega_put}, Theta: {theta_put}, Rho: {rho_put}")


# Call the function to execute the calculations
financial_calculations_input()
